## Overview

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sachin0034/MLAI-community-labs/blob/main/Class-Labs/Lab-14%28LLM-Judge-lab%29/Lab-4%28Continuously%20Evaluate%20your%20AI%20agents%20-%20Azure%20AI%20Foundry%29/Continuously_Evaluate_your_AI_agents_Azure_AI_Foundry.ipynb)


In this lab, you'll explore **Continuous Evaluation for Agents** — a powerful tool that provides near real-time evualteand monitoring for your AI applications. Through hands-on exercises, you'll learn how to implement, configure, and leverage this feature to ensure optimal agent performance.

## Introduction

Continuous Evaluation enables your AI agents to be evaluate monitored at a defined sampling rate. It provides actionable insights into quality, safety, and performance, with all metrics surfaced in the **Foundry Observability Dashboard**. Evaluations are directly linked to traces, making it easy to troubleshoot issues, perform root cause analysis, and continuously improve your AI workflows.

## Key Benefits

- **Near Real-Time Monitoring**: Immediate visibility into agent performance
- **Quality Assurance**: Continuous evaluation of interaction quality
- **Safety Monitoring**: Ongoing tracking of safety-related metrics
- **Performance Optimization**: Data-driven insights for enhancing agent efficiency
- **Early Issue Detection**: Proactive identification of issues before user impact
- **Root Cause Analysis**: Trace-linked evaluations for detailed debugging


# Getting Started

## Prerequisites

**1 Azure Subscription**: You must have an **Azure subscription with access to Azure OpenAI Service**. A premium subscription is strongly recommended to access all features.

**2 Azure AI Foundry Project**: You need an active AI Foundry project. If you don't have one, you can create one using this guide:

* **3 Create a Foundry Project**: [https://learn.microsoft.com/en-us/azure/ai-foundry/how-to/create-projects](https://learn.microsoft.com/en-us/azure/ai-foundry/how-to/create-projects?pivots=fdp-project&tabs=ai-foundry)



**4 Application Insights**: To monitor your application and view evaluation dashboards, you must connect Application Insights.
   - Navigate to your project in **Azure AI Foundry**.
   - Select **Monitoring** on the left-hand menu and go to **Application Analytics**.
   - Connect your **Application Insights** resource to the project.

**5 Dowload the Dummy MSA Document From Here For Testing** [Click Here](https://drive.google.com/file/d/1E557kdNBZ5cDUvVDLNrEVRuKcRSYDG3Z/view?usp=sharing)

## Sign into Azure interactively using the Azure CLI

To interact with Azure resources directly from your Colab notebook, you first need to install the **Azure CLI** in the Colab environment. Use the command below to install it:
**bold text**


In [ ]:
!curl -sL https://aka.ms/InstallAzureCLIDeb | sudo bash

## Interactive login
`az login` is a command used with the **Azure CLI (Command Line Interface)** to authenticate and connect your local environment (like Terminal, Command Prompt, or Jupyter Notebook) to your **Microsoft Azure account**.

## Requirements

- ✅ You must have an **Azure subscription account**.
- ✅ **Azure CLI** should be installed (`az` command should work in your terminal or notebook).

## When you run `az login`, you will receive a message like:
***To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code SHH9K2G35 to authenticate.***

***Click the link, paste the provided code, and follow the login steps to complete authentication.***

## Step-by-step Instructions

1. Copy and open this link in your browser:  
   👉 [https://microsoft.com/devicelogin](https://microsoft.com/devicelogin)

2. Paste the **code** displayed in your terminal (e.g., `SHH9K2G35`) and click **Next**. ⚠️ The code shown will be different each time.

3. Sign in using your **Microsoft Azure account credentials**.

4. After signing in, you'll be asked to **select your Azure subscription** if you have more than one.

5. Once selected, authentication is complete, and you're **ready to use Azure CLI** commands.

In [4]:
!az login

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code AVCA9BL47 to authenticate.

Retrieving tenants and subscriptions for the selection...

[Tenant and subscription selection]

No     Subscription name              Subscription ID                       Tenant
-----  -----------------------------  ------------------------------------  -----------
[1] *  Microsoft Azure LegalGraph AI  0d74045d-252c-4b44-856a-71c5dee6b80a  Pragyaa LLC

The default is marked with an *; the default tenant is 'Pragyaa LLC' and subscription is 'Microsoft Azure LegalGraph AI' (0d74045d-252c-4b44-856a-71c5dee6b80a).

Select a subscription and tenant (Type a number or Enter for no changes): 1

Tenant: Pragyaa LLC
Subscription: Microsoft Azure LegalGraph AI (0d74045d-252c-4b44-856a-71c5dee6b80a)

[Announcements]
With the new Azure CLI login experience, you can select the subscription you want to use more easily. Learn more about it and its configuration at https://go

## Step 2: Install Required Packages

Ok, so now your **first step** is to install the required Python packages. These will enable you to work with **Azure AI services**, handle **document processing**, and build an interactive UI using **Gradio**.

Run the following command in your environment:

| Package Name        | Description                                                                 |
|---------------------|-----------------------------------------------------------------------------|
| `azure-ai-projects` | SDK to interact with Azure AI projects and services                         |
| `azure-identity`    | Provides Azure Active Directory token authentication for Azure SDKs         |
| `python-dotenv`     | Loads environment variables from a `.env` file into the environment          |
| `gradio`            | Builds simple and interactive web UIs for ML models and functions            |
| `PyPDF2`            | Extracts text, metadata, and handles PDF file reading and manipulation       |
| `python-docx`       | Reads, writes, and manipulates Microsoft Word `.docx` files                  |
| `openpyxl`          | Reads and writes Excel `.xlsx` files                                         |
| `pandas`            | Provides data structures and tools for data manipulation and analysis        |


In [ ]:
# Install all required packages for Azure AI, document processing, and Gradio interface
! pip install azure-ai-projects azure-identity python-dotenv gradio PyPDF2 python-docx openpyxl pandas

## Step3: Import Required Libraries

Once all packages are installed, import the necessary libraries for:

- Working with Azure AI Projects
- Document processing (PDF, DOCX, Excel)
- Building the Gradio interface
- Loading environment variables

In [ ]:
import os
import time
import io
import pandas as pd
import gradio as gr
from typing import Optional, Tuple, Dict

# Azure AI imports
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import (
    AgentEvaluationRequest,
    InputDataset,
    EvaluatorIds,
    EvaluatorConfiguration,
    AgentEvaluationSamplingConfiguration,
    AgentEvaluationRedactionConfiguration,
)

# Document processing imports
import PyPDF2
from docx import Document


print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


## Step 4 : Configure Azure AI Project
Next, configure your Azure AI project by setting the required `endpoint` and `model_deployment_name`. These should be replaced with **your actual Azure AI project details**.
- Create an Enpoint Azure AI Foundry Guide  : [Click Here](https://learn.microsoft.com/en-us/azure/ai-foundry/how-to/develop/sdk-overview?pivots=programming-language-csharp
)
- Deploy AI Models in Azure AI Foundry Guide - [Click Here](https://www.linkedin.com/pulse/how-deploy-ai-models-minutes-azure-foundryguide-pradip-tivhale-dpbjc
)

In [43]:
import os
# You would typically need these imports for the actual client initialization
# from azure.identity import DefaultAzureCredential
# from azure.ai.projects import AIProjectClient

# Azure AI Project configuration
# Replace these with your actual Azure AI project details
# Get the Keys from Here : https://learn.microsoft.com/en-us/azure/ai-foundry/how-to/develop/sdk-overview?pivots=programming-language-csharp
endpoint = "INSERT YOUR KEYS HERE"
# Deploy AI Models in Azure AI Foundry Guide : https://www.linkedin.com/pulse/how-deploy-ai-models-minutes-azure-foundryguide-pradip-tivhale-dpbjc
model_deployment_name = "INSERT YOUR KEYS HERE"

# Global variables to store agent and project client
project_client = None
agent = None
extracted_text = ""

try:
    print("🚀 Initializing Azure AI configuration...")

    # 1. Check if placeholder keys are replaced
    if "INSERT YOUR KEYS HERE" in (endpoint, model_deployment_name):
        raise ValueError("Please replace the placeholder values for 'endpoint' and 'model_deployment_name'.")

    # 2. The actual client initialization would go here.
    #    This is where errors related to authentication or invalid endpoints would be caught.
    #    Example:
    #    credential = DefaultAzureCredential()
    #    project_client = AIProjectClient(endpoint=endpoint, credential=credential)

    print("✅ Azure AI configuration is valid.")

except ValueError as ve:
    # Catches specific configuration errors, like the placeholder issue.
    print(f"❌ Configuration Error: {ve}")

except Exception as e:
    # Catches any other exceptions during initialization (e.g., connection, authentication).
    print(f"❌ An unexpected error occurred during initialization: {e}")

🚀 Initializing Azure AI configuration...
❌ Configuration Error: Please replace the placeholder values for 'endpoint' and 'model_deployment_name'.


## Step 5: Define Document Extraction Functions
This function is responsible for **extracting text** from documents you upload.  
It supports major file types like **PDF**, **DOCX**, and **TXT** formats.

---

### What It Does this function do :

- Reads the uploaded file
- Detects the file type
- Extracts the text content accordingly
- Returns the clean extracted text

In [8]:
def extract_text_from_pdf(file_path: str) -> str:
    """
    Extract text content from PDF files
    Args:
        file_path (str): Path to the PDF file
    Returns:
        str: Extracted text content
    """
    try:
        text = ""
        with open(file_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            for page in pdf_reader.pages:
                text += page.extract_text() + "\n"
        return text.strip()
    except Exception as e:
        return f"Error extracting PDF: {str(e)}"

def extract_text_from_docx(file_path: str) -> str:
    """
    Extract text content from Word documents
    Args:
        file_path (str): Path to the DOCX file
    Returns:
        str: Extracted text content
    """
    try:
        doc = Document(file_path)
        text = ""
        for paragraph in doc.paragraphs:
            text += paragraph.text + "\n"
        return text.strip()
    except Exception as e:
        return f"Error extracting DOCX: {str(e)}"

def extract_text_from_txt(file_path: str) -> str:
    """
    Extract text content from plain text files
    Args:
        file_path (str): Path to the TXT file
    Returns:
        str: Extracted text content
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read().strip()
    except Exception as e:
        return f"Error extracting TXT: {str(e)}"

def extract_text_from_document(file_path: str) -> str:
    """
    Main function to extract text from various document formats
    Args:
        file_path (str): Path to the document file
    Returns:
        str: Extracted text content
    """
    if not file_path:
        return "No file uploaded"

    file_extension = os.path.splitext(file_path.lower())[1]

    if file_extension == '.pdf':
        return extract_text_from_pdf(file_path)
    elif file_extension == '.docx':
        return extract_text_from_docx(file_path)
    elif file_extension == '.txt':
        return extract_text_from_txt(file_path)
    else:
        return f"Unsupported file format: {file_extension}"

print("📄 Document extraction functions defined!")

📄 Document extraction functions defined!


## Step 6 : Define Azure AI Agent

Below are the two main functions used to initialize and create an Azure AI agent that can work with uploaded document content.

---

### 1. `initialize_azure_client()`

**Purpose:**  
Authenticates your environment with Azure and returns an instance of the `AIProjectClient`, allowing you to interact with Azure AI services.  
It uses `DefaultAzureCredential` for seamless login, including browser-based login if needed.

---

### 2. `create_agent_with_document_context(document_text: str)`

**Purpose:**  
Creates a custom AI agent in Azure using the extracted document text as part of its instruction prompt.  
This agent can answer questions specifically based on the document's content or fall back to general knowledge when needed.  
It ensures responses are clear about whether the information came from the uploaded document or from external knowledge.


In [9]:
def initialize_azure_client():
    """
    Initialize Azure AI Project Client with authentication
    Returns:
        AIProjectClient: Authenticated Azure AI project client
    """
    try:
        credential = DefaultAzureCredential(exclude_interactive_browser_credential=False)
        client = AIProjectClient(endpoint=endpoint, credential=credential)
        return client
    except Exception as e:
        print(f"❌ Error initializing Azure client: {str(e)}")
        return None

def create_agent_with_document_context(document_text: str):
    """
    Create an Azure AI agent with document context in instructions
    Args:
        document_text (str): Extracted text from uploaded document
    Returns:
        tuple: (agent_object, success_message)
    """
    global project_client, agent

    try:
        # Initialize client if not already done
        if project_client is None:
            project_client = initialize_azure_client()
            if project_client is None:
                return None, "❌ Failed to initialize Azure client"

        # Create agent with document context
        instructions = f"""You are a helpful AI assistant with access to the following document content:

DOCUMENT CONTENT:
{document_text}

Instructions:
- Use the provided document content to answer user questions accurately
- If the question relates to the document, provide specific information from it
- If the question is outside the document scope, provide general helpful responses
- Always be clear about whether your answer comes from the document or general knowledge
"""

        agent = project_client.agents.create_agent(
            model=model_deployment_name,
            name="document-assistant",
            instructions=instructions,
        )

        return agent, f"✅ Agent created successfully with ID: {agent.id}"

    except Exception as e:
        return None, f"❌ Error creating agent: {str(e)}"

print("🤖 Azure AI agent functions defined!")


🤖 Azure AI agent functions defined!


## Step 7 : User Query Handling & AI Evaluation

### 3. `process_user_query(user_query: str)`

**Purpose:**  
Processes a user's question using the Azure AI agent and returns a detailed answer along with a simple evaluation summary.

This function performs the following:
- Starts a new thread and sends the user’s query.
- Runs the Azure AI agent on the query.
- Waits for and retrieves the assistant's response.
- Submits the response for automated evaluation based on metrics like:
  - **Violence** (safety check)
  - **Fluency** (language quality)
  - **Relevance** (context accuracy)
- Returns the AI's answer and a status table showing that evaluation was submitted to Azure.

> 📊 Detailed evaluation metrics and scores will be available inside **Azure AI Studio**.


In [35]:
def process_user_query(user_query: str) -> Tuple[str, pd.DataFrame]:
    """
    Process user query through Azure AI agent and return response with evaluations
    Args:
        user_query (str): User's question/query
    Returns:
        tuple: (response_text, simple_status_dataframe)
    """
    global project_client, agent

    if not project_client or not agent:
        return "❌ Agent not initialized. Please upload a document first.", pd.DataFrame()

    try:
        # Create a new thread for conversation
        thread = project_client.agents.threads.create()
        print(f"Created thread, thread ID: {thread.id}")

        # Create user message
        message = project_client.agents.messages.create(
            thread_id=thread.id,
            role="user",
            content=user_query
        )
        print(f"Created message, message ID: {message.id}")

        # Create and run the agent
        run = project_client.agents.runs.create(
            thread_id=thread.id,
            agent_id=agent.id
        )

        # Poll the run as long as run status is queued or in progress - EXACTLY like your original code
        while run.status in ["queued", "in_progress", "requires_action"]:
            # Wait for a second
            time.sleep(1)
            run = project_client.agents.runs.get(thread_id=thread.id, run_id=run.id)
            print(f"Run status: {run.status}")

        # List messages - EXACTLY like your original code
        messages = project_client.agents.messages.list(thread_id=thread.id)
        response_text = ""

        for msg in messages:
            print(f"Role: {msg.role}, Content: {msg.content}")
            if msg.role == "assistant":
                response_text = msg.content[0].text.value if msg.content else "No response"

        # Create evaluation request - EXACTLY like your original code
        agent_evaluation_request = AgentEvaluationRequest(
            run_id=run.id,
            thread_id=thread.id,

           evaluators={
            "violence": EvaluatorConfiguration(
                id=EvaluatorIds.VIOLENCE,
            ),
            "fluency": EvaluatorConfiguration(
                id=EvaluatorIds.FLUENCY,
            ),
            "relevance": EvaluatorConfiguration(
                id=EvaluatorIds.RELEVANCE,
            ),
            "coherence": EvaluatorConfiguration(
                id=EvaluatorIds.COHERENCE,
            ),
            },

            sampling_configuration=AgentEvaluationSamplingConfiguration(
                name="test",
                sampling_percent=100,
                max_request_rate=100,
            ),
            redaction_configuration=AgentEvaluationRedactionConfiguration(
                redact_score_properties=False,
            ),
            app_insights_connection_string=project_client.telemetry.get_connection_string(),
        )

        # Create evaluation - EXACTLY like your original code
        agent_evaluation_response = project_client.evaluations.create_agent_evaluation(
            evaluation=agent_evaluation_request
        )

        # Print evaluation response - EXACTLY like your original code
        print(agent_evaluation_response)

        # Simple status table for Gradio interface (metrics will show in Azure dashboard)
        eval_data = {
            # This list has 4 items
            "Metric": ["Violence", "Fluency", "Relevance","Coherence"],
            "Status": ["✅ Submitted to Azure", "✅ Submitted to Azure", "✅ Submitted to Azure","✅ Submitted to Azure"],
            "Note": ["Check Azure AI Studio", "Check Azure AI Studio", "Check Azure AI Studio","Check Azure AI Studio"]
        }
        eval_df = pd.DataFrame(eval_data)

        return response_text, eval_df

    except Exception as e:
        print(f"Error in process_user_query: {str(e)}")
        error_df = pd.DataFrame({"Error": [str(e)]})
        return f"❌ Error processing query: {str(e)}", error_df

print("💬 Chat and evaluation functions defined!")

💬 Chat and evaluation functions defined!


## Step 8 :  Gradio File & Chat Interaction Handlers

### `handle_file_upload(file)`

**Purpose**:  
Handles file uploads from the Gradio UI.  
It performs the following steps:
- Validates the file.
- Extracts text from the uploaded document (PDF, DOCX, or TXT).
- Creates an Azure AI agent with the document content as its knowledge base.
- Returns a preview of the document text, a success/error message, and agent creation status.

---

### `handle_chat_query(query, chat_history)`

**Purpose**:  
Handles the user’s chat query using the previously created Azure agent.  
It performs the following steps:
- Validates the input query.
- Sends the query to the Azure agent and waits for a response.
- Appends the Q&A pair to the chat history.
- Returns the updated history and evaluation results.


In [36]:
def handle_file_upload(file):
    """
    Handle document upload and extract text content
    Args:
        file: Gradio file upload object
    Returns:
        tuple: (extracted_text, status_message, agent_status)
    """
    global extracted_text

    if file is None:
        return "", "❌ No file uploaded", "Agent not ready"

    try:
        # Extract text from uploaded document
        extracted_text = extract_text_from_document(file.name)

        if extracted_text.startswith("Error") or extracted_text.startswith("Unsupported"):
            return extracted_text, extracted_text, "Agent not ready"

        # Create agent with document context
        agent_obj, agent_status = create_agent_with_document_context(extracted_text)

        if agent_obj:
            preview_text = extracted_text
            return preview_text, "✅ Document processed successfully!", agent_status
        else:
            return extracted_text, agent_status, "Agent creation failed"

    except Exception as e:
        error_msg = f"❌ Error processing file: {str(e)}"
        return "", error_msg, "Agent not ready"

def handle_chat_query(query, chat_history):
    """
    Handle user chat query and update conversation history
    Args:
        query (str): User's question
        chat_history (list): Previous conversation history
    Returns:
        tuple: (updated_chat_history, empty_query, evaluation_dataframe)
    """
    if not query.strip():
        return chat_history, "", pd.DataFrame()

    # Process the query
    response, eval_df = process_user_query(query)

    # Update chat history
    chat_history.append([query, response])

    return chat_history, "", eval_df

print("🎛️ Gradio interface functions defined!")

🎛️ Gradio interface functions defined!


## Step 9 : Gradio Interface Overview

The create_gradio_interface function defines and configures a user interface using Gradio for an Azure AI Document Chat Assistant. This assistant allows users to upload documents and interact with an AI that can answer questions based on the document content. Here's what the interface includes:

### Interface Sections
**1. Document Upload Section**
- Allows users to upload files in PDF, DOCX, or TXT format.
- Displays upload and agent readiness status.
- Shows a preview of the extracted text from the uploaded document.

**2. Chat Section**
- Enables users to chat with the AI assistant after uploading a document.
- Includes a chat window to view conversations.
- Provides a text input and Send button for submitting questions.

**3. Evaluation Metrics**
- Shows automated evaluation results for each AI response
- Metrics include:
    - Violence: Ensures responses are safe.
    - Fluency: Checks how well-written the response is.
    - Relevance: Measures how closely the response matches the user's query.

**4. Usage Instructions**
- Provides a step-by-step guide on how to use the app.
- Describes what each evaluation metric means.




In [37]:
def create_gradio_interface():
    """
    Create and configure the Gradio interface for the document chat application
    Returns:
        gr.Blocks: Configured Gradio interface
    """

    with gr.Blocks(title="Azure AI Document Chat Assistant", theme=gr.themes.Soft()) as interface:

        gr.Markdown("""
        # 🤖 Azure AI Document Chat Assistant

        Upload a document and chat with an AI assistant that has knowledge of your document content.
        Get real-time evaluation metrics for safety, fluency, and relevance.
        """)

        with gr.Row():
            with gr.Column(scale=1):
                # Document Upload Section
                gr.Markdown("## 📄 Step 1: Upload Document")
                file_upload = gr.File(
                    label="Upload Document (PDF, DOCX, TXT)",
                    file_types=[".pdf", ".docx", ".txt"]
                )

                upload_status = gr.Textbox(
                    label="Upload Status",
                    value="No file uploaded",
                    interactive=False
                )

                agent_status = gr.Textbox(
                    label="Agent Status",
                    value="Agent not ready",
                    interactive=False
                )

                # Document Preview
                gr.Markdown("## 👀 Document Preview")
                document_preview = gr.Textbox(
                    label="Extracted Text Preview",
                    lines=8,
                    interactive=False,
                    placeholder="Upload a document to see preview..."
                )

            with gr.Column(scale=2):
                # Chat Section
                gr.Markdown("## 💬 Step 2: Chat with Your Document")

                chatbot = gr.Chatbot(
                    label="Conversation",
                    height=400,
                    placeholder="Upload a document first, then start chatting!"
                )

                with gr.Row():
                    query_input = gr.Textbox(
                        label="Your Question",
                        placeholder="Ask a question about your document...",
                        scale=4
                    )
                    send_button = gr.Button("Send", variant="primary", scale=1)

                # Evaluation Results Section
                gr.Markdown("## 📊 Step 3: Evaluation Metrics")
                evaluation_table = gr.Dataframe(
                    label="Response Evaluation Scores",
                    headers=["Metric", "Score", "Status"],
                    interactive=False
                )

        # Event handlers
        file_upload.change(
            fn=handle_file_upload,
            inputs=[file_upload],
            outputs=[document_preview, upload_status, agent_status]
        )

        send_button.click(
            fn=handle_chat_query,
            inputs=[query_input, chatbot],
            outputs=[chatbot, query_input, evaluation_table]
        )

        query_input.submit(
            fn=handle_chat_query,
            inputs=[query_input, chatbot],
            outputs=[chatbot, query_input, evaluation_table]
        )

        # Example usage section
        gr.Markdown("""
        ## 📝 How to Use:
        1. **Upload**: Select and upload a document (PDF, DOCX, or TXT)
        2. **Wait**: Wait for the document to be processed and agent to be created
        3. **Chat**: Ask questions about your document content
        4. **Review**: Check the evaluation metrics for each response

        ## 🔍 Evaluation Metrics:
        - **Violence**: Checks for violent or harmful content
        - **Fluency**: Measures how well-written and coherent the response is
        - **Relevance**: Evaluates how relevant the response is to your query
        """)

    return interface

print("🎨 Gradio interface creation function defined!")

🎨 Gradio interface creation function defined!


## Step 10: Launching the Gradio Interface

This section of the code is responsible for creating and starting the Gradio application for the **Azure AI Document Chat Assistant**.

Once you are done and started interacting with your assistant, **you can now monitor your model’s performance** in the Azure AI Studio:

### 📊 Metric Evaluator

After launching your Gradio app and sending some queries, head over to **Azure AI Studio > Monitoring > Evaluation**.  
Here, you'll find detailed **evaluation metrics** based on your requests:


![Metric Evaluator](./images/img-1.png)

---

### ⚙️ Resource Usage

To keep track of your model’s consumption and performance, navigate to **Azure AI Studio > Monitoring > Resource Usage**.  
This dashboard helps you visualize how many requests have been made and the token usage breakdown.

![Resource Usage](./images/img-2.png)

---

With this, you can analyze how efficiently your model is operating and optimize your prompts or responses as needed.


In [ ]:
# Create and launch the Gradio interface
print("🚀 Creating Gradio interface...")
app = create_gradio_interface()

print("✅ Interface created successfully!")
print("🌐 Launching application...")

# Launch the interface
# Use share=True to create a public link, or share=False for local only
app.launch(
    share=True,  # Set to False if you don't want a public link
    server_name="0.0.0.0",  # Allow external connections in Colab
    server_port=7860,  # Port number
    debug=True  # Enable debug mode
)
